# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import json
import requests
import random as rand
import time

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame.
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data.
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,32.76,74,100,18.72,TF,1727975852
1,1,puerto ayora,-0.7393,-90.3518,70.56,96,97,5.99,EC,1727975853
2,2,bethel,41.3712,-73.4140,71.02,75,100,5.75,US,1727975854
3,3,puerto natales,-51.7236,-72.4875,48.65,49,20,41.43,CL,1727975855
4,4,monrovia,6.3005,-10.7969,79.66,79,78,10.56,LR,1727975857


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# I want to make the map look nice... random colors.
city_data_df['color'] = pd.Series()

for index, row in city_data_df.iterrows():
    city_data_df.loc[index, 'color'] = '#%06x' % rand.randint(0, 0xFFFFFF)
    
# Configure the map plot.
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    s = 'Humidity',
    geo = True,
    tiles = True,
    color = 'color',
    alpha = .5,
    hover_cols = 'City',
    frame_height = 500,
    frame_width = 700
)

# Get rid of the color columns now...
city_data_df = city_data_df.drop(columns = ['color'])

# Display the map.
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (color,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values.
mask = (city_data_df['Max Temp'] >= 55) & (city_data_df['Max Temp'] < 65) & (city_data_df['Humidity'] <= 50)
ideal_df = city_data_df.loc[mask, :].reset_index(drop = True)

# Drop any rows with null values.
ideal_df = ideal_df.dropna(how = 'any')

# Display sample data.
display(ideal_df.info())
display(ideal_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     23 non-null     int64  
 1   City        23 non-null     object 
 2   Lat         23 non-null     float64
 3   Lng         23 non-null     float64
 4   Max Temp    23 non-null     float64
 5   Humidity    23 non-null     int64  
 6   Cloudiness  23 non-null     int64  
 7   Wind Speed  23 non-null     float64
 8   Country     23 non-null     object 
 9   Date        23 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 1.9+ KB


None

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,28,minot air force base,48.4209,-101.3391,57.38,38,0,17.27,US,1727975885
1,32,paso de los toros,-32.8167,-56.5167,60.67,47,16,6.80,UY,1727975890
2,114,dom pedrito,-30.9828,-54.6731,61.56,48,31,9.04,BR,1727975988
3,143,concepcion del oro,24.6333,-101.4167,61.05,39,12,2.28,MX,1727976022
4,151,casper,42.8666,-106.3131,58.71,38,0,5.01,US,1727976031


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = pd.Series()

# Display sample data
display(hotel_df.info())
display(hotel_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        23 non-null     object 
 1   Country     23 non-null     object 
 2   Lat         23 non-null     float64
 3   Lng         23 non-null     float64
 4   Humidity    23 non-null     int64  
 5   Hotel Name  0 non-null      object 
dtypes: float64(2), int64(1), object(3)
memory usage: 1.2+ KB


None

,City,Country,Lat,Lng,Humidity,Hotel Name
0,minot air force base,US,48.4209,-101.3391,38,NaN
1,paso de los toros,UY,-32.8167,-56.5167,47,NaN
2,dom pedrito,BR,-30.9828,-54.6731,48,NaN
3,concepcion del oro,MX,24.6333,-101.4167,39,NaN
4,casper,US,42.8666,-106.3131,38,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Define your API key
geoapify_key = 'f4dfb7e2243740e28780e188a2807d20'  # Replace with your actual API key


# Make a function to search for the hotel.
def hotel_lookup(lat, lon):
    # Set up parameters for search.
    base_url = "https://api.geoapify.com/v2/places"
    radius = 10000
    format = 'json'
    categories = 'accommodation.hotel'
    # conditions = ''
    limit = 20
    filter = f'circle:{lon},{lat},{radius}'
    bias = f'proximity:{lon},{lat}'
    
    params = {
        'apiKey': geoapify_key,
        'format': format,
        'categories': categories,
        # 'conditions': conditions,
        'limit': limit,
        'filter': filter,
        'bias': bias
    }

    # Make a request.
    resp = requests.get(base_url, params)

    # Check status code of request.
    if resp.status_code == 200:
        # Successful request! Let's get that data as a JSON.
        data = resp.json()
    
    # Failed request... that's not good.
    else:
        print(f'Failed to make a request for latitude: {lat} and longitude: {lon}')
        return 'No Hotel Found'

    # Let's get that name.
    try:
        # Get past the 'features' portion of the request.
        data = data.get('features')

        # Loop thru our results until we find something valid.
        for hotel in data:
            # Grab the name of the hotel from the result.
            # If this fails: it returns as 'None'
            name = hotel.get('properties', {}).get('name', None)

            # Success! We found a name for the hotel.
            # Let's return that and stop searching.
            if name != None:
                return name

            # This hotel didn't return us a name.
            # We pass to the next result from our search.
            else:
                pass

    # We get some unusable response from our request.
    # Notify the user and pass to the next iteration.
    except (KeyError, IndexError) as e:
        print(f'Error: {e}')
        print(f'Something went wrong when finding a hotel for latitude: {lat} and longitude: {lon}.')
        pass

    # Nothing was found if we made it down here.
    return 'No Hotel Found'

In [12]:
print('Beginning hotel search...')
# Iterate through the hotel_df DataFrame.
for index, row in hotel_df.iterrows():
    # Run our 'hotel_lookup' command for this index and put it in the data frame.
    hotel_df.loc[index, 'Hotel Name'] = hotel_lookup(hotel_df.loc[index, 'Lat'], 
                                                     hotel_df.loc[index, 'Lng'])
        
    # Log the search results.
    print(f"{hotel_df.loc[index, 'City']}, {hotel_df.loc[index, 'Country']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Wait a second for the API's sake.
    time.sleep(1)

# Confirm the search is over.
print('Finished searching for hotels.')
print('--------------------------------------------------')

# Display sample data.
display(hotel_df.info())
display(hotel_df.head())

Beginning hotel search...
minot air force base, US - nearest hotel: No Hotel Found
paso de los toros, UY - nearest hotel: Hotel Italiana
dom pedrito, BR - nearest hotel: Hotel Alexandre
concepcion del oro, MX - nearest hotel: No Hotel Found
casper, US - nearest hotel: Days Inn by Wyndham Casper
malayer, IR - nearest hotel: خانه معلم
yashkul', RU - nearest hotel: No Hotel Found
rawson, AR - nearest hotel: Hotel Deportivo
ilinge, ZA - nearest hotel: No Hotel Found
necochea, AR - nearest hotel: Cabañas Río Quequén
alice springs, AU - nearest hotel: Aurora Alice Springs
queenstown, ZA - nearest hotel: Queens Hotel
lypova dolyna, UA - nearest hotel: Готель
moyynkum, KZ - nearest hotel: No Hotel Found
hasanbeyli, TR - nearest hotel: Bahçe Park Otel
carahue, CL - nearest hotel: No Hotel Found
mount isa, AU - nearest hotel: Ibis Styles
kyzylorda, KZ - nearest hotel: Нур
rikaze, CN - nearest hotel: Gesar Hotel
comodoro rivadavia, AR - nearest hotel: Victoria
hancheng, CN - nearest hotel: No Hot

None

,City,Country,Lat,Lng,Humidity,Hotel Name
0,minot air force base,US,48.4209,-101.3391,38,No Hotel Found
1,paso de los toros,UY,-32.8167,-56.5167,47,Hotel Italiana
2,dom pedrito,BR,-30.9828,-54.6731,48,Hotel Alexandre
3,concepcion del oro,MX,24.6333,-101.4167,39,No Hotel Found
4,casper,US,42.8666,-106.3131,38,Days Inn by Wyndham Casper


In [13]:
display(hotel_df.head(23))

,City,Country,Lat,Lng,Humidity,Hotel Name
0,minot air force base,US,48.4209,-101.3391,38,No Hotel Found
1,paso de los toros,UY,-32.8167,-56.5167,47,Hotel Italiana
2,dom pedrito,BR,-30.9828,-54.6731,48,Hotel Alexandre
3,concepcion del oro,MX,24.6333,-101.4167,39,No Hotel Found
4,casper,US,42.8666,-106.3131,38,Days Inn by Wyndham Casper
5,malayer,IR,34.2969,48.8235,27,خانه معلم
6,yashkul',RU,46.1711,45.3435,34,No Hotel Found
7,rawson,AR,-43.3002,-65.1023,20,Hotel Deportivo
8,ilinge,ZA,-31.9768,27.0427,45,No Hotel Found
9,necochea,AR,-38.5473,-58.7368,45,Cabañas Río Quequén


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# I want to make the map look nice... random colors.
hotel_df['color'] = pd.Series()

for index, row in hotel_df.iterrows():
    hotel_df.loc[index, 'color'] = '#%06x' % rand.randint(0, 0xFFFFFF)
    
# Configure the map plot.
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    color = 'color',
    hover_cols = ['Hotel Name', 'Country', 'City'],
    frame_height = 500,
    frame_width = 700
)

# Get rid of the color columns now...
hotel_df = hotel_df.drop(columns = ['color'])

# Display the map.
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (color,Hotel Name,Country,City)